# This Notebook Is To Inspect Optuna's HTML Results

In [ ]:
import optuna

# Checkout which optuna study you want to investigate
storage = "sqlite:///optuna_chain_isotope_study.db"
study_summaries = optuna.get_all_study_summaries(storage=storage)

print("Available studies:")
for summary in study_summaries:
    print(f"  - Study name: {summary.study_name}")
    print(f"    Direction: {summary.direction}")
    print(f"    N trials: {summary.n_trials}")
    print(f"    Best value: {summary.best_trial.value if summary.best_trial else 'N/A'}")
    print()

In [ ]:
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_slice,
    plot_contour,
    plot_parallel_coordinate,
    plot_edf,
    plot_intermediate_values,
    plot_pareto_front
)
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Load your study
study = optuna.load_study(
    study_name="isotope_DNN_optimization",
    storage="sqlite:///optuna_chain_isotope_study.db"
)

# ============================================
# 1. BASIC STUDY INFORMATION
# ============================================
print(f"Study name: {study.study_name}")
print(f"Direction: {study.direction}")
print(f"Number of trials: {len(study.trials)}")
print(f"Best trial: {study.best_trial.number}")
print(f"Best value: {study.best_value}")
print(f"Best params: {study.best_params}")

# ============================================
# 2. TRIAL DATA ANALYSIS
# ============================================
# Convert to DataFrame for easy analysis
df = study.trials_dataframe()
display(df.head())

# Statistical summary
print("\n=== Statistics ===")
print(df['value'].describe())

# Check for failed/pruned trials
print(f"\nComplete trials: {len(df[df['state'] == 'COMPLETE'])}")
print(f"Pruned trials: {len(df[df['state'] == 'PRUNED'])}")
print(f"Failed trials: {len(df[df['state'] == 'FAIL'])}")

# ============================================
# 3. CONVERGENCE ANALYSIS
# ============================================
fig = plot_optimization_history(study)
fig.show()

# Custom convergence plot with running best
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['number'],
    y=df['value'],
    mode='markers',
    name='Trial value',
    marker=dict(size=5, opacity=0.5)
))

# Running best
running_best = df['value'].cummin() if study.direction == optuna.study.StudyDirection.MINIMIZE else df['value'].cummax()
fig.add_trace(go.Scatter(
    x=df['number'],
    y=running_best,
    mode='lines',
    name='Best so far',
    line=dict(color='red', width=2)
  ))
fig.update_layout(title='Optimization Progress', xaxis_title='Trial', yaxis_title='Objective Value')
fig.show()

# ============================================
# 4. PARAMETER IMPORTANCE
# ============================================
fig = plot_param_importances(study)
fig.show()

# Get importance values
importances = optuna.importance.get_param_importances(study)
print("\n=== Parameter Importances ===")
for param, importance in sorted(importances.items(), key=lambda x: x[1], reverse=True):
    print(f"{param}: {importance:.4f}")

# ============================================
# 5. PARAMETER RELATIONSHIPS
# ============================================
# Slice plot (already have HTML, but can regenerate)
fig = plot_slice(study)
fig.show()

# Contour plot for pairwise relationships
fig = plot_contour(study)
fig.show()

# Parallel coordinate plot
fig = plot_parallel_coordinate(study)
fig.show()

# ============================================
# 6. DETAILED PARAMETER ANALYSIS
# ============================================
params = list(study.best_params.keys())

# Correlation between parameters and objective
for param in params:
    param_col = f'params_{param}'
    if param_col in df.columns:
        corr = df[param_col].corr(df['value'])
        print(f"Correlation {param} <-> objective: {corr:.4f}")

# Parameter distributions for best N trials
top_n = 10
best_trials = df.nsmallest(top_n, 'value') if study.direction == optuna.study.StudyDirection.MINIMIZE else df.nlargest(top_n, 'value')
print(f"\n=== Top {top_n} trials ===")
display(best_trials[['number', 'value'] + [f'params_{p}' for p in params]])

# ============================================
# 7. EMPIRICAL DISTRIBUTION FUNCTION
# ============================================
fig = plot_edf(study)
fig.show()

# ============================================
# 8. INTERMEDIATE VALUES (for pruned trials)
# ============================================
if any(trial.intermediate_values for trial in study.trials):
    fig = plot_intermediate_values(study)
    fig.show()

# ============================================
# 9. CUSTOM ANALYSIS
# ============================================
# Time analysis
if 'datetime_start' in df.columns:
    df['duration'] = (df['datetime_complete'] - df['datetime_start']).dt.total_seconds()
    print(f"\n=== Duration Statistics ===")
    print(f"Mean: {df['duration'].mean():.2f}s")
    print(f"Median: {df['duration'].median():.2f}s")
    print(f"Total: {df['duration'].sum()/3600:.2f}h")

# Parameter value ranges in successful trials
print("\n=== Parameter ranges in best trials ===")
for param in params:
    param_col = f'params_{param}'
    if param_col in best_trials.columns:
        print(f"{param}: [{best_trials[param_col].min():.4f}, {best_trials[param_col].max():.4f}]")

# ============================================
# 10. EXPORT FOR FURTHER ANALYSIS
# ============================================
# Save complete results
df.to_csv('optuna_results_full.csv', index=False)

# Save best parameters
pd.DataFrame([study.best_params]).to_csv('best_params.csv', index=False)

# Save top N trials
best_trials.to_csv(f'top_{top_n}_trials.csv', index=False)

In [ ]:
# Individual trial inspection
trial = study.best_trial
print(f"Trial number: {trial.number}")
print(f"Value: {trial.value}")
print(f"Params: {trial.params}")
print(f"User attrs: {trial.user_attrs}")
print(f"System attrs: {trial.system_attrs}")

# Check if specific parameters interact
import scipy.stats as stats
for i, p1 in enumerate(params):
    for p2 in params[i+1:]:
        col1, col2 = f'params_{p1}', f'params_{p2}'
        if col1 in df.columns and col2 in df.columns:
            corr, pval = stats.pearsonr(df[col1].dropna(), df[col2].dropna())
            if abs(corr) > 0.5:  # Strong correlation
                print(f"Strong correlation between {p1} and {p2}: {corr:.3f} (p={pval:.4f})")

In [ ]:
import optuna
import yaml
from pprint import pprint

# Load the study
storage = "sqlite:///optuna_Pu241_study.db"
study_name = "isotope_DNN_optimization"  # Replace with your study name

study = optuna.load_study(study_name=study_name, storage=storage)

# Get best trial
best_trial = study.best_trial

print(f"{'='*60}")
print(f"BEST TRIAL SUMMARY")
print(f"{'='*60}")
print(f"Trial number: {best_trial.number}")
print(f"Best value (loss): {best_trial.value:.6f}")
print(f"Duration: {best_trial.duration}")
print(f"\n{'='*60}")
print(f"BEST HYPERPARAMETERS")
print(f"{'='*60}")

# Print parameters in a readable format
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

print(f"\n{'='*60}")
print(f"YAML FORMAT (copy-paste ready)")
print(f"{'='*60}")

# Convert to YAML format
best_params_yaml = yaml.dump(best_trial.params, default_flow_style=False, sort_keys=False)
print(best_params_yaml)

# ============================================
# OPTION 1: Save directly to a YAML file
# ============================================
with open('best_hyperparameters.yaml', 'w') as f:
    yaml.dump(best_trial.params, f, default_flow_style=False, sort_keys=False)
print(f"\n✓ Saved to: best_hyperparameters.yaml")